In [39]:
import numpy as np 
import matplotlib as mp

In [40]:
class Sphere:
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 4 * np.pi * self.radius * self.radius

In [41]:
# hello world 
sphere: Sphere = Sphere(1)
assert sphere.area() == 4 * np.pi, "make sure this runs"
print("sphere area:", sphere.area())

sphere area: 12.566370614359172
